# Suma de Vectores

In [ ]:
#Creamos el archivo .cpp
%%writefile SumaVectores.cpp
/*
PROGRAMA:            SumaVectores_OpenMP.cpp
DESCRIPCIÓN:         Suma dos vectores con tipo de dato float y almacena el resultado
                     en un tercer vector del mismo tipo.
			               Paralelizamos la suma usando directivas de OpenMP.
AUTOR:               Francisco J. Hernández-López
Fecha Inicial:       16/Mar/2016
Fecha Actualización: 19/Oct/2023 Probándo el código en Google Colab
*/

#include <stdio.h>
#include <stdlib.h>
#include <iostream>
#include <math.h>
#include <time.h>
//OpenMP
#include <omp.h>

void Suma_vectores(float *c, float *a, float *b, long int N)
{
	long int i;
#pragma omp parallel shared(a,b,c,N) private(i)
	{
#pragma omp for
		for (i = 0; i<N; i++){
			//c[i] = a[i] + b[i];
			c[i] = cos(a[i]) + (int)(b[i])%3;
		}
	}
}

// Código principal que se ejecuta en el Host
int main(void){
	float *a_h, *b_h, *c_h; //Punteros a arreglos en el Host
	long int N = 1000000000; //Número de elementos en los arreglos
	long int i;
	//struct timeb start, end;
	double t_ini, t_fin, time_cpu_seconds;

	int diff;
	size_t size_float = N * sizeof(float);
	a_h = (float *)malloc(size_float); // Pedimos memoria en el Host
	b_h = (float *)malloc(size_float);
	c_h = (float *)malloc(size_float); //También se puede con cudaMallocHost

	omp_set_num_threads(2); //Fijamos cuantos hilos vamos a lanzar

	//Inicializamos los arreglos a,b en el Host
/*#pragma omp parallel shared(a_h,b_h,N) private(i)
	{
#pragma omp for*/
		for (i = 0; i<N; i++){
			a_h[i] = (float)i;
			b_h[i] = (float)(i + 1);
		}
	//}

	printf("\nArreglo a:\n");
	for (i=0; i<20; i++) printf("%f ", a_h[i]);
	printf("\n\nArreglo b:\n");
	for (i=0; i<20; i++) printf("%f ", b_h[i]);

	printf("\n\nComenzando el procesamiento...");
	t_ini = clock();
	Suma_vectores(c_h, a_h, b_h, N);
	t_fin = clock();
	time_cpu_seconds = (t_fin - t_ini) / CLOCKS_PER_SEC;

	//Resultado
	printf("\n\nArreglo c:\n");
	for (i = 0; i<20; i++)
		printf("%f ", c_h[i]);

	printf("\nProcesamiento Finalizado...");
	printf("\nTiempo de procesamiento en CPU: %lf segundos.\n", time_cpu_seconds);

	// Liberamos la memoria del Host
	free(a_h);
	free(b_h);
	free(c_h);
	return(0);
}

Overwriting SumaVectores.cpp


In [ ]:
# Compilamos el programa
! g++ SumaVectores.cpp -o SumaVectores_secuencial

In [ ]:
# Ejecutamos el programa
! ./SumaVectores_secuencial


Arreglo a:
0.000000 1.000000 2.000000 3.000000 4.000000 5.000000 6.000000 7.000000 8.000000 9.000000 10.000000 11.000000 12.000000 13.000000 14.000000 15.000000 16.000000 17.000000 18.000000 19.000000 

Arreglo b:
1.000000 2.000000 3.000000 4.000000 5.000000 6.000000 7.000000 8.000000 9.000000 10.000000 11.000000 12.000000 13.000000 14.000000 15.000000 16.000000 17.000000 18.000000 19.000000 20.000000 

Comenzando el procesamiento...

Arreglo c:
2.000000 2.540302 -0.416147 0.010008 1.346356 0.283662 1.960170 2.753902 -0.145500 0.088870 1.160928 0.004426 1.843854 2.907447 0.136737 0.240312 1.042341 -0.275163 1.660317 2.988705 
Procesamiento Finalizado...
Tiempo de procesamiento en CPU: 18.768764 segundos.


In [ ]:
# Compilamos el programa
! g++ -fopenmp SumaVectores.cpp -o SumaVectores_paralelo

In [ ]:
# Ejecutamos el programa
! ./SumaVectores_paralelo


Arreglo a:
0.000000 1.000000 2.000000 3.000000 4.000000 5.000000 6.000000 7.000000 8.000000 9.000000 10.000000 11.000000 12.000000 13.000000 14.000000 15.000000 16.000000 17.000000 18.000000 19.000000 

Arreglo b:
1.000000 2.000000 3.000000 4.000000 5.000000 6.000000 7.000000 8.000000 9.000000 10.000000 11.000000 12.000000 13.000000 14.000000 15.000000 16.000000 17.000000 18.000000 19.000000 20.000000 

Comenzando el procesamiento...

Arreglo c:
2.000000 2.540302 -0.416147 0.010008 1.346356 0.283662 1.960170 2.753902 -0.145500 0.088870 1.160928 0.004426 1.843854 2.907447 0.136737 0.240312 1.042341 -0.275163 1.660317 2.988705 
Procesamiento Finalizado...
Tiempo de procesamiento en CPU: 26.820239 segundos.


In [ ]:
! lscpu

Architecture:            x86_64
  CPU op-mode(s):        32-bit, 64-bit
  Address sizes:         46 bits physical, 48 bits virtual
  Byte Order:            Little Endian
CPU(s):                  2
  On-line CPU(s) list:   0,1
Vendor ID:               GenuineIntel
  Model name:            Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:          6
    Model:               85
    Thread(s) per core:  2
    Core(s) per socket:  1
    Socket(s):           1
    Stepping:            3
    BogoMIPS:            4000.32
    Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clf
                         lush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_
                         good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fm
                         a cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hyp
                         ervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd i

# Mejor hay que probar el ejemplo en su propia computadora...